###### Needed installation

In [13]:
pip install flatten_json

Note: you may need to restart the kernel to use updated packages.


##### Main libraries

In [52]:
#basic libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#scrapping libraries

import requests, json
from flatten_json import flatten # to simplify nested json files in pandas df

In [53]:
pd.set_option('max_colwidth', None)

##### 1) Scrapping upcoming events

In [294]:
# upcoming events

url = "https://betsapi2.p.rapidapi.com/v1/bet365/upcoming"

querystring = {"sport_id":"1","league_id":"10041110","page":"1"}

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

###### 1.1) Upcoming events : from json to dataframe

In [295]:
upcoming_events=json.loads(response.text[67:-1])

response_columns = ['id','sport_id','time','time_status','league_id','league_name','home_id','home_name','away_id','away_name','ss','our_event_id','r_id','updated_at']
df_upc_events = pd.DataFrame(columns=response_columns)

for match in upcoming_events:
    match_list = flatten(match)
    df_upc_events = df_upc_events.append(match_list, ignore_index=True)

df_upc_events = df_upc_events.drop(['away_id', 'home_id','league_id','league_name','r_id','ss','time', 'time_status','sport_id','our_event_id','updated_at'], axis = 1)
df_upc_events = df_upc_events[0:10]

##### 2) Scrapping upcoming events odds

In [333]:
url = "https://betsapi2.p.rapidapi.com/v3/bet365/prematch"

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

df_upc_odds = pd.DataFrame(columns = ['id','1','X','2','Both_Scores_Y','Both_Scores_N'])

for i in df_upc_events.id:

    querystring = {"FI":i}
    
    upcoming_odds = requests.request("GET", url, headers=headers, params=querystring)
    
    # turn output into a daframe
    
    upcoming_odds =json.loads(upcoming_odds.text[24:-2])                                    # Fitting the output
    upcoming_odds = flatten(upcoming_odds)

    FI = upcoming_odds.get('FI')
    result_1 = upcoming_odds.get('main_sp_full_time_result_odds_0_odds')                    
    result_X = upcoming_odds.get('main_sp_full_time_result_odds_1_odds')
    result_2 = upcoming_odds.get('main_sp_full_time_result_odds_2_odds')                    # Retrieving key data
    result_both_score_yes = upcoming_odds.get('goals_sp_both_teams_to_score_odds_0_odds')
    result_both_score_no = upcoming_odds.get('goals_sp_both_teams_to_score_odds_1_odds')
    
    list = [FI,result_1,result_X, result_2, result_both_score_yes, result_both_score_no]       
    a_series = pd.Series(list, index = df_upc_odds.columns)                                 # Gathering key data 
    df_upc_odds = df_upc_odds.append(a_series, ignore_index=True)

##### 3) Merging both datasets

In [336]:
df_odds = df_upc_events.merge(df_upc_odds, how = 'left', on = 'id')

In [337]:
df_odds

,id,home_name,away_name,1,X,2,Both_Scores_Y,Both_Scores_N
0,99186569,Valencia,Villarreal,3.800,3.300,2.000,1.800,1.950
1,99186571,Valladolid,Getafe,2.900,2.800,2.800,2.250,1.571
2,99186576,Elche,Sevilla,7.000,4.000,1.500,2.200,1.615
3,99186585,Cadiz,Eibar,3.250,3.100,2.350,2.100,1.666
4,99186590,Osasuna,Barcelona,5.500,4.333,1.533,1.666,2.100
5,99186592,Huesca,Celta Vigo,2.900,3.300,2.400,1.950,1.800
6,99186595,Atletico Madrid,Real Madrid,2.700,3.300,2.625,1.909,1.909
7,99186598,Real Sociedad,Levante,1.500,4.000,7.000,1.800,1.950
8,99186604,Athletic Bilbao,Granada,1.650,3.800,5.250,2.050,1.700
9,99186609,Real Betis,CD Alaves,1.750,3.700,4.500,1.950,1.800
